In [464]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [465]:
pd.set_option('display.max_columns', None)
flats = pd.read_csv('~/DS_Bootcamp/FlatPrices/data/_data.csv',index_col=0)
IDs = flats.iloc[:,0]
IDs = IDs.rename('ID')

In [466]:
#Первый признак - комнаты, разобьем их на категории: числовое количество комнат (0 - не определено) и one-hot кодированием признаков смежности или изолированности 

#Заполним число комнат - nan нулями, разбив во вторую колонку данные по комнатам 
rooms = pd.DataFrame(flats['Количество комнат'].fillna('0').map(lambda x: map(str.strip,x.split(','))).to_list(),columns=['Num_Rooms','Room_Type'])

#get_dummies разобьет на три колонки из которых приведем к двум логической алгеброй
RoomDummie = pd.get_dummies(rooms['Room_Type'])
rooms['Num_Rooms'] = rooms['Num_Rooms'].map(int)
RoomDummie['Изолированная'] = RoomDummie['Изолированная'] + RoomDummie['Оба варианта']
RoomDummie['Смежная'] = RoomDummie['Смежная'] + RoomDummie['Оба варианта']

#applymap оказывается будет deprecated
RoomDummie = RoomDummie.drop(columns='Оба варианта').map(int).rename(columns={'Изолированная':'has_isolated','Смежная':'has_adjacent'})
RoomsFinal = pd.concat([rooms,RoomDummie],axis=1).drop(columns='Room_Type')
RoomsFinal.head(10)

,Num_Rooms,has_isolated,has_adjacent
0,4,0,0
1,4,0,0
2,4,1,1
3,4,1,1
4,2,0,0
5,3,0,0
6,5,1,1
7,3,0,0
8,3,0,0
9,4,0,0


In [467]:
flats = pd.concat([RoomsFinal,flats.loc[:,'Тип':]],axis=1)
#Убрали ID, уберем остальные колонки не имеющие числового смысла
dropcolumns = ['Тип','Телефоны','Описание'] #тип только квартиры -нет ни нан ни других типов
flats = flats.drop(columns=dropcolumns,axis=1)

#Переименуем
colforrename = flats.columns[3:10]
flats = flats.rename(columns=dict(zip(colforrename,['Time_to_subway','Adress','Square_m2','HouseType','Parking','Price','Renovation'])))


In [468]:
#Обработка времени до метро - nan и none заменены -1, остальные числами
flats['Time_to_subway'] = flats['Time_to_subway'].fillna('-1').map(lambda x: re.search(r'(\d+) мин',x).group(1) if re.search(r'(\d+) мин',x) else '-1').astype(int)


In [469]:
#Обработка площади - берем общую, в ремонте заменяем nan
flats['Square_m2'] = flats['Square_m2'].str.split('/').map(lambda x: x[0]).astype('float')
flats['Renovation'] = flats['Renovation'].fillna('не указано')


## Пока таблица выглядит следующим образом

In [470]:
#Пока что просто заменим nan парковки на "не указано"
flats['Parking'] = flats['Parking'].fillna('не указано')
flats.head(10)

,Num_Rooms,has_isolated,has_adjacent,Time_to_subway,Adress,Square_m2,HouseType,Parking,Price,Renovation,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,4,0,0,9,"Москва, улица Новый Арбат, 27",200.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157
1,4,0,0,8,"Москва, улица Новый Арбат, 27",198.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126
2,4,1,1,7,"Москва, улица Новый Арбат, 27",200.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086
3,4,1,1,3,"Москва, переулок Плотников, 21С1",170.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456
4,2,0,0,7,"Москва, улица Новый Арбат, 15",58.0,"12/26, Панельный",не указано,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.9,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615
5,3,0,0,5,"Москва, 1-й Смоленский переулок, 21",92.0,3/7,не указано,"470000.0 руб./ За месяц, Залог - 470000 руб., ...",Дизайнерский,NaN,NaN,NaN,NaN,NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Smolensky De Luxe,NaN,NaN,NaN,NaN,https://www.cian.ru/rent/flat/274837728
6,5,1,1,4,"Москва, переулок Романов, 3С1",213.0,4/5,наземная,"350000.0 руб./ За месяц, Залог - 350000 руб., ...",Евроремонт,43-25-25-25-22,NaN,На улицу и двор,Совмещенный (2),"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Сти...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/273643908
7,3,0,0,3,"Москва, улица Арбат, 43С3",98.0,"2/4, Монолитный",подземная,"250000.0 руб./ За месяц, Залог - 250000 руб., ...",Евроремонт,26 22 15,NaN,Во двор,"Совмещенный (1), Раздельный (1)","Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),Нет,https://www.cian.ru/rent/flat/274475342
8,3,0,0,9,"Москва, Новинский бульвар, 18С1",120.0,"5/10, Сталинский",открытая,"130000.0 руб./ За месяц, Залог - 130000 руб., ...",Евроремонт,45 25 25,NaN,На улицу,Совмещенный (1),Можно с животными,"Мебель на кухне, Ванна, Стиральная машина, Кон...",NaN,NaN,3.0,Пасс (1),Нет,https://www.cian.ru/rent/flat/273973191
9,4,0,0,10,"Москва, улица Арбат, 30/3С1",90.0,"2/7, Сталинский",открытая,"210000.0 руб./ За месяц, Залог - 210000 руб., ...",Евроремонт,16 14 16 20,Балкон (1),На улицу и двор,"Совмещенный (1), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Сти...",NaN,NaN,310.0,Пасс (1),Да,https://www.cian.ru/rent/flat/272900409


In [471]:
#pd.set_option('display.max_colwidth',None)
#Для облегчения использования regex разобьем на списки, уберем пробелы и снова соединим с ;
flats['Price'] = flats['Price'].str.split(r'\.?,|\./|/').map(lambda x: ';'.join(map(str.strip, x)) + ';')
flats['Price'].head(10)

0    500000.0 руб;За месяц;Залог - 500000 руб;Комму...
1    500000.0 руб;За месяц;Залог - 500000 руб;Комму...
2    500000.0 руб;За месяц;Залог - 500000 руб;Срок ...
3    400000.0 руб;За месяц;Залог - 400000 руб;Срок ...
4    225000.0 руб;За месяц;Залог - 225000 руб;Комму...
5    470000.0 руб;За месяц;Залог - 470000 руб;Срок ...
6    350000.0 руб;За месяц;Залог - 350000 руб;Срок ...
7    250000.0 руб;За месяц;Залог - 250000 руб;Комму...
8    130000.0 руб;За месяц;Залог - 130000 руб;Комму...
9    210000.0 руб;За месяц;Залог - 210000 руб;Комму...
Name: Price, dtype: object

In [472]:
#Напишем регулярное выражение
def rg(x):
    return re.search(r'(?P<rent>\d+)\.\d (?P<curr>.+?);За (?P<period>\w+);(Сумма коммунальных платежей - (?P<commsumm>\d+\.\d) \2;)?(Залог - (?P<deposit>\d+) \2;)?(Коммунальные услуги (?P<comminc>\w+);)?(Срок аренды - (?P<renttime>.+?);)?(Предоплата (?P<prepayment>\d+) мес;)?',x).groupdict()
Price = flats['Price'].map(rg)
Price = pd.DataFrame(Price.to_list())
Price = pd.concat([flats['Price'], Price],axis=1)
Price


,Price,rent,curr,period,commsumm,deposit,comminc,renttime,prepayment
0,500000.0 руб;За месяц;Залог - 500000 руб;Комму...,500000,руб,месяц,None,500000,включены,Длительный,1
1,500000.0 руб;За месяц;Залог - 500000 руб;Комму...,500000,руб,месяц,None,500000,включены,Длительный,1
2,500000.0 руб;За месяц;Залог - 500000 руб;Срок ...,500000,руб,месяц,None,500000,None,Длительный,1
3,400000.0 руб;За месяц;Залог - 400000 руб;Срок ...,400000,руб,месяц,None,400000,None,Длительный,1
4,225000.0 руб;За месяц;Залог - 225000 руб;Комму...,225000,руб,месяц,None,225000,включены,Длительный,1
...,...,...,...,...,...,...,...,...,...
23363,42000.0 руб;За месяц;Залог - 42000 руб;Коммуна...,42000,руб,месяц,None,42000,включены,Длительный,1
23364,45000.0 руб;За месяц;Залог - 45000 руб;Коммуна...,45000,руб,месяц,None,45000,включены,Длительный,1
23365,50000.0 руб;За месяц;Залог - 50000 руб;Коммуна...,50000,руб,месяц,None,50000,включены,Длительный,1
23366,55000.0 руб;За месяц;Залог - 50000 руб;Коммуна...,55000,руб,месяц,None,50000,включены,Длительный,2


In [473]:
#Проверить корректность отработки выражения можно таким путем
#Базовые проверки на наличие только числовых или nan значений
print(Price['curr'].unique(),Price['rent'].astype(int).sum(),Price['deposit'].fillna(0).astype(int).sum())

#Правильность захвата данных группами можно проверить фильтруя таблицу по точно имеющимся совпадениям,
#но там, где соответствующая группа выдала в результате none - таких строк быть не должно
Price[(Price['deposit'].isna()) & (Price['Price'].map(lambda x: bool(re.search('Залог',x))))]

['руб' '$' '€'] 1937070234 11726799628


,Price,rent,curr,period,commsumm,deposit,comminc,renttime,prepayment


In [474]:
#Обработаем таблицу цен
for col in ['rent','deposit']:
    Price[col] = Price[col].fillna(0).astype(float)

Price['prepayment'] = Price['prepayment'].fillna(0).astype(int)

doll_curr = 90
euro_curr = 98

Price.loc[Price['curr'] == '$','rent'] *= doll_curr
Price.loc[Price['curr'] == '€','rent'] *= euro_curr

Price.loc[Price['curr'] == '$','deposit'] *= doll_curr
Price.loc[Price['curr'] == '€','deposit'] *= euro_curr

#После конвертации валют колонка валют уже не нужна, период тоже так как он везде месяц
Price = Price.drop(['curr','period'],axis=1)

In [475]:
Price.head(10)

,Price,rent,commsumm,deposit,comminc,renttime,prepayment
0,500000.0 руб;За месяц;Залог - 500000 руб;Комму...,500000.0,None,500000.0,включены,Длительный,1
1,500000.0 руб;За месяц;Залог - 500000 руб;Комму...,500000.0,None,500000.0,включены,Длительный,1
2,500000.0 руб;За месяц;Залог - 500000 руб;Срок ...,500000.0,None,500000.0,None,Длительный,1
3,400000.0 руб;За месяц;Залог - 400000 руб;Срок ...,400000.0,None,400000.0,None,Длительный,1
4,225000.0 руб;За месяц;Залог - 225000 руб;Комму...,225000.0,None,225000.0,включены,Длительный,1
5,470000.0 руб;За месяц;Залог - 470000 руб;Срок ...,470000.0,None,470000.0,None,Длительный,2
6,350000.0 руб;За месяц;Залог - 350000 руб;Срок ...,350000.0,None,350000.0,None,Длительный,1
7,250000.0 руб;За месяц;Залог - 250000 руб;Комму...,250000.0,None,250000.0,включены,Длительный,1
8,130000.0 руб;За месяц;Залог - 130000 руб;Комму...,130000.0,None,130000.0,включены,Длительный,1
9,210000.0 руб;За месяц;Залог - 210000 руб;Комму...,210000.0,None,210000.0,включены,Длительный,1


In [476]:
#Предположу что корректно закодировать 1 и 0 т.к. на аренда на длительный срок предпочительнее
rtd = {'Длительный':1,'На несколько месяцев':0}
Price['renttime'] = Price['renttime'].map(lambda x: rtd[x])

In [477]:
#Кодируем коммунальные платежи
Price['comminc'] = Price['comminc'].fillna(0)
#Price[(~Price['commsumm'].isna()) & (~Price['comminc'].isna())]
Price.loc[Price['comminc'] == 'включены','commsumm'] = 0
Price.loc[Price['comminc'] == 'включены','comminc'] = 1
Price[['rent','commsumm','deposit']] = Price[['rent','commsumm','deposit']].fillna(0).astype(float)
Price['comminc'] = Price['comminc'].fillna(0).astype(int)
Price = Price.drop(columns='Price')

/tmp/ipykernel_3734/3707681058.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Price['comminc'] = Price['comminc'].fillna(0).astype(int)


In [478]:
flats = pd.concat([flats.loc[:,'Num_Rooms':'Parking'],Price,flats.loc[:,'Renovation':]],axis=1)

In [479]:
#разбиваем дом на этажи и тип дома
HouseParams = flats['HouseType'].str.split('[/,]',regex=True,expand=True).rename(columns=dict(zip(range(3),['Flat_floor','Max_floors','HouseType'])))
HouseParams['HouseType'] = HouseParams['HouseType'].fillna('Не указано')
HouseParams[['Flat_floor','Max_floors']] = HouseParams[['Flat_floor','Max_floors']].astype(int)
HouseParams.head()

,Flat_floor,Max_floors,HouseType
0,5,16,Монолитный
1,5,16,Монолитно-кирпичный
2,5,16,Не указано
3,5,6,Не указано
4,12,26,Панельный


In [480]:
flats = pd.concat([flats.loc[:,'Num_Rooms':'Square_m2'],HouseParams,flats.loc[:,'Parking':]],axis=1)
flats.head()

,Num_Rooms,has_isolated,has_adjacent,Time_to_subway,Adress,Square_m2,Flat_floor,Max_floors,HouseType,Parking,rent,commsumm,deposit,comminc,renttime,prepayment,Renovation,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,4,0,0,9,"Москва, улица Новый Арбат, 27",200.0,5,16,Монолитный,подземная,500000.0,0.0,500000.0,1,1,1,Дизайнерский,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157
1,4,0,0,8,"Москва, улица Новый Арбат, 27",198.0,5,16,Монолитно-кирпичный,подземная,500000.0,0.0,500000.0,1,1,1,Дизайнерский,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126
2,4,1,1,7,"Москва, улица Новый Арбат, 27",200.0,5,16,Не указано,подземная,500000.0,0.0,500000.0,0,1,1,Евроремонт,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086
3,4,1,1,3,"Москва, переулок Плотников, 21С1",170.0,5,6,Не указано,подземная,400000.0,0.0,400000.0,0,1,1,Евроремонт,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456
4,2,0,0,7,"Москва, улица Новый Арбат, 15",58.0,12,26,Панельный,не указано,225000.0,0.0,225000.0,1,1,1,Евроремонт,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.9,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615


In [481]:
#Target encoding для типа дома, парковке и ремонту
for col in ['HouseType','Parking','Renovation']:
    flats[col + 'Enc'] = flats.groupby(col)['rent'].transform('mean')
    flats[col] = flats[col + 'Enc'].map(lambda x: round(x, 2))
    flats = flats.drop(columns=col + 'Enc')
flats.head()

,Num_Rooms,has_isolated,has_adjacent,Time_to_subway,Adress,Square_m2,Flat_floor,Max_floors,HouseType,Parking,rent,commsumm,deposit,comminc,renttime,prepayment,Renovation,"Площадь комнат, м2",Балкон,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,4,0,0,9,"Москва, улица Новый Арбат, 27",200.0,5,16,106839.39,167618.14,500000.0,0.0,500000.0,1,1,1,173624.09,NaN,NaN,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157
1,4,0,0,8,"Москва, улица Новый Арбат, 27",198.0,5,16,151494.07,167618.14,500000.0,0.0,500000.0,1,1,1,173624.09,25 25 20 25,NaN,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126
2,4,1,1,7,"Москва, улица Новый Арбат, 27",200.0,5,16,124970.37,167618.14,500000.0,0.0,500000.0,0,1,1,84298.80,NaN,NaN,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086
3,4,1,1,3,"Москва, переулок Плотников, 21С1",170.0,5,6,124970.37,167618.14,400000.0,0.0,400000.0,0,1,1,84298.80,14-42-20-19,NaN,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456
4,2,0,0,7,"Москва, улица Новый Арбат, 15",58.0,12,26,45290.64,75981.08,225000.0,0.0,225000.0,1,1,1,84298.80,20 18,NaN,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.9,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615


In [482]:
#Судя по всему склеарн классная вещь, еще бы понимать что-то, а не просто код копировать
from sklearn.preprocessing import StandardScaler

# Extract numerical features
num_features = ['rent','deposit']
X = flats[num_features]

# Normalize data using StandardScaler
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Print normalized data
norm = pd.DataFrame(X_normalized,columns=num_features)

In [483]:
#Финальная сборка моей части таблицы и запись в csv
flats[['rent','deposit']] = norm[['rent','deposit']]
flats = flats.drop(columns='Adress') # увы у меня нет АПИ ключа яндекс карт да и знаний чтобы написать нормальный осознанный код с http запросом
myPart = pd.concat([IDs,flats.loc[:,'Num_Rooms':'Renovation']],axis=1)
myPart.head()

,ID,Num_Rooms,has_isolated,has_adjacent,Time_to_subway,Square_m2,Flat_floor,Max_floors,HouseType,Parking,rent,commsumm,deposit,comminc,renttime,prepayment,Renovation
0,271271157,4,0,0,9,200.0,5,16,106839.39,167618.14,3.245532,0.0,-0.000045,1,1,1,173624.09
1,271634126,4,0,0,8,198.0,5,16,151494.07,167618.14,3.245532,0.0,-0.000045,1,1,1,173624.09
2,271173086,4,1,1,7,200.0,5,16,124970.37,167618.14,3.245532,0.0,-0.000045,0,1,1,84298.80
3,272197456,4,1,1,3,170.0,5,6,124970.37,167618.14,2.465349,0.0,-0.001608,0,1,1,84298.80
4,273614615,2,0,0,7,58.0,12,26,45290.64,75981.08,1.100028,0.0,-0.004344,1,1,1,84298.80


In [484]:
myPart.to_csv('~/DS_Bootcamp/FlatPrices/myData.csv',index=True)

In [485]:
# Rent = lambda x: re.search(r'(?P<rent>\d+)\.\d руб\./',x)
# Period = lambda x: re.search(r'/ За (?P<period>\w+),',x)
# CommSumm = lambda x: re.search(r'Сумма коммунальных платежей - (?P<commsumm>\d+\.\d) руб\.',x)
# Deposit = lambda x: re.search(r'Залог - (?P<deposit>\d+) руб\.,',x)
# CommIncluded = lambda x: re.search(r'Коммунальные услуги (?P<comminc>\w+),',x)
# RentTime = lambda x: re.search(r'Срок аренды - (.+),|Срок аренды - (.+)$',x)
# Prepayment = lambda x: re.search(r'Предоплата (?P<prepayment>\d+) мес',x)

# Price = flats['Цена']
# funcdict = {'rent':Rent,'period':Period,'commsumm':CommSumm,'deposit':Deposit,'comminc':CommIncluded,'renttime':RentTime,'prepayment':Prepayment}
# reslist = []
# for elem in Price:
#     res_dict = {}
#     for key,value in funcdict.items():
#         res_dict[key] = value(elem).group(1) if value(elem) else None 
#     reslist.append(res_dict)

# Price = pd.concat([Price,pd.DataFrame(reslist)],axis=1)
#Price[(Price['renttime'].isna()) & (Price['Цена'].map(lambda x: bool(re.search('Срок аренды',x))))]

# i = 0
# for elem in flats['Цена']:
#     try:
#         print(i, RentPeriod(elem), CommSumm(elem))
#     except:
#         print(i, elem)
#     i += 1